In [1]:
from queue import Queue
import numpy as np

In [2]:
class Graph:
    def __init__(self, n, m, graph_type):
        self.adj_list_ = None
        self.nodes_amount_ = None

        if graph_type == 'rectengular':
            self._create_rectengular(n, m)


    def _create_rectengular(self, n, m):
        self.adj_list_ = {}
        idx = lambda r, t: r*m + t 

        self.nodes_amount_ = n*m  

        for i in range(0, n):
            for j in range(0, m):
                # Corner case 
                current_idx = idx(i, j) 

                neighbours = []
                if i - 1 >= 0:
                     neighbours.append(idx(i-1, j))
                if i + 1 < n:
                     neighbours.append(idx(i+1, j))
                if j - 1 >= 0:
                     neighbours.append(idx(i, j-1))
                if j + 1 < m:
                     neighbours.append(idx(i, j+1))
                
                self.adj_list_[current_idx] = neighbours
    

    def delete_vertex(self, v):
        neighbours = self.adj_list_[v]

        for neighbour in neighbours:
            neighbour_list = self.adj_list_[neighbour]
            neighbour_list.remove(v)
        
        del self.adj_list_[v]
        self.nodes_amount_ -= 1
    

    def delete_edge(self, e, v):
        self.adj_list_[e].remove(v)
        self.adj_list_[v].remove(e)
    

    def shortest_pairwise_path(self):
        # Calculate all pairwise distances using bfs and return distance matrix
        path_distance = np.zeros(shape=(self.nodes_amount_, self.nodes_amount_))

        vertex_to_idx = {vertex: idx for idx, vertex in enumerate(self.adj_list_)}

        for current_vertex in self.adj_list_:
            parent_vertices = self._bfs(current_vertex)
            for vertex in parent_vertices:
                cur_vertex_idx =  vertex_to_idx[current_vertex]
                vertex_idx = vertex_to_idx[vertex]

                if path_distance[vertex_idx, cur_vertex_idx] != 0:
                    # Symmetric matrix
                    path_distance[cur_vertex_idx, vertex_idx] = path_distance[vertex_idx, cur_vertex_idx]
                    continue

                distance = 0
                prev = vertex
                
                while prev != current_vertex:
                    prev = parent_vertices[prev]
                    distance += 1
                    
                path_distance[cur_vertex_idx, vertex_idx] = distance

        return path_distance
    
    
    def _bfs(self, vertex):
        queue = Queue()
        visited = {key: False for key in self.adj_list_}
        prev = {}

        queue.put(vertex)
        visited[vertex] = True
        
        while not queue.empty():
            node = queue.get()
            neighbours = self.adj_list_[node]

            for neighbour in neighbours:
                if not visited[neighbour]:
                    queue.put(neighbour)
                    visited[neighbour] = True
                    prev[neighbour] = node
        return prev

In [3]:
graph = Graph(3, 4, 'rectengular')

In [4]:
graph.adj_list_

{0: [4, 1],
 1: [5, 0, 2],
 2: [6, 1, 3],
 3: [7, 2],
 4: [0, 8, 5],
 5: [1, 9, 4, 6],
 6: [2, 10, 5, 7],
 7: [3, 11, 6],
 8: [4, 9],
 9: [5, 8, 10],
 10: [6, 9, 11],
 11: [7, 10]}

In [6]:
graph.delete_vertex(5)

In [7]:
graph.adj_list_

{0: [4, 1],
 1: [0, 2],
 2: [6, 1, 3],
 3: [7, 2],
 4: [0, 8],
 6: [2, 10, 7],
 7: [3, 11, 6],
 8: [4, 9],
 9: [8, 10],
 10: [6, 9, 11],
 11: [7, 10]}

In [8]:
graph.delete_edge(10, 11)

In [9]:
graph.delete_edge(0, 1)

In [10]:
graph.adj_list_

{0: [4],
 1: [2],
 2: [6, 1, 3],
 3: [7, 2],
 4: [0, 8],
 6: [2, 10, 7],
 7: [3, 11, 6],
 8: [4, 9],
 9: [8, 10],
 10: [6, 9],
 11: [7]}

In [11]:
graph._bfs(0)

{4: 0, 8: 4, 9: 8, 10: 9, 6: 10, 2: 6, 7: 6, 1: 2, 3: 2, 11: 7}

In [12]:
shortest_paths = graph.shortest_pairwise_path()

In [13]:
shortest_paths

array([[0., 7., 6., 7., 1., 5., 6., 2., 3., 4., 7.],
       [7., 0., 1., 2., 6., 2., 3., 5., 4., 3., 4.],
       [6., 1., 0., 1., 5., 1., 2., 4., 3., 2., 3.],
       [7., 2., 1., 0., 6., 2., 1., 5., 4., 3., 2.],
       [1., 6., 5., 6., 0., 4., 5., 1., 2., 3., 6.],
       [5., 2., 1., 2., 4., 0., 1., 3., 2., 1., 2.],
       [6., 3., 2., 1., 5., 1., 0., 4., 3., 2., 1.],
       [2., 5., 4., 5., 1., 3., 4., 0., 1., 2., 5.],
       [3., 4., 3., 4., 2., 2., 3., 1., 0., 1., 4.],
       [4., 3., 2., 3., 3., 1., 2., 2., 1., 0., 3.],
       [7., 4., 3., 2., 6., 2., 1., 5., 4., 3., 0.]])